In [1]:
import pandas as pd
import re
import yaml
import sqlparse
import os
import pandas as pd
import numpy as np
import requests
from IPython.display import display, Markdown

from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

In [2]:
def add_repo_root_path():
    import os
    import sys
    repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
    if repo_root not in sys.path:
        sys.path.append(repo_root)
        
add_repo_root_path()
from src import generate_knowledge
from src import create_rag_db
from src import llm_chain_tools
from src.enhanced_retriever import EnhancedRetriever

In [3]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 50)
#pd.set_option('display.width', None)
#pd.set_option('display.max_colwidth', 10) 

In [4]:
generate_knowledge.add_repo_root_path()
import openai_setup

OPENAI_API_KEY = openai_setup.conf['key']
OPENAI_PROJECT = openai_setup.conf['project']
OPENAI_ORGANIZATION = openai_setup.conf['organization']
DEFAULT_LLM_MODEL = "gpt-4o-mini"
CHROMADB_DIRECTORY = '../chromadb'
COLLECTION_NAME = "my_chromadb" 

import os
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ['OPENAI_MODEL_NAME'] = DEFAULT_LLM_MODEL

In [5]:
langchain_openai_embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY, model="text-embedding-ada-002")
langchain_openai_llm = ChatOpenAI(model=DEFAULT_LLM_MODEL, temperature=0.1, openai_api_key=OPENAI_API_KEY, openai_organization = OPENAI_ORGANIZATION)

In [24]:
from langchain_openai import ChatOpenAI

loaded_vectorstore = Chroma(
    collection_name=COLLECTION_NAME,
    persist_directory=CHROMADB_DIRECTORY,
    embedding_function=langchain_openai_embeddings
)

_, repo_name = generate_knowledge.extract_owner_and_repo('https://github.com/dbt-labs/jaffle-shop')
dbt_models_df = pd.read_csv('../data/dbt_models_' + repo_name + '.csv')
dbt_project_df = pd.read_csv('../data/dbt_project_' + repo_name + '.csv')
dbt_repo_knowledge_df = create_rag_db.merge_dbt_models_and_project_dfs(dbt_models_df, dbt_project_df)

import nest_asyncio
nest_asyncio.apply()

import importlib
import src.llm_agents_flow
importlib.reload(src.llm_agents_flow)
from src.llm_agents_flow import dbtChatFlow

files = {
    'agents': '../config/agents.yml',
    'tasks': '../config/tasks.yml'
}

In [25]:
flow = dbtChatFlow(files)
flow.plot()

user_input = "I want to add a new column 'overdue' to the model orders that come from raw_orders source, and have it available in customers. the overdue column is directly available in raw_orders, is not necessairy to calcylate it "
result = flow.kickoff(inputs={"request": user_input, "dbt_repo_knowledge_df": dbt_repo_knowledge_df, "vectorstore": loaded_vectorstore, "embedding_function":langchain_openai_embeddings})
display(Markdown(f"<div style='font-size: 18px;'><b>User input:</b> <i>{user_input}</i></div><hr>"))
display(Markdown(result.raw))

Overriding of current TracerProvider is not allowed
Overriding of current TracerProvider is not allowed
Overriding of current TracerProvider is not allowed
Overriding of current TracerProvider is not allowed
Overriding of current TracerProvider is not allowed
Overriding of current TracerProvider is not allowed
Overriding of current TracerProvider is not allowed
Overriding of current TracerProvider is not allowed
Overriding of current TracerProvider is not allowed


Plot saved as crewai_flow.html
# Agent: Identify if the user's request explicitly mentions a specific model for retrieving information or implementing changes.
## Task: Verify if the request explicitly mentions a model that requires information retrieval or changes. Request: I want to add a new column 'overdue' to the model orders that come from raw_orders source, and have it available in customers. the overdue column is directly available in raw_orders, is not necessairy to calcylate it  Current dbt lineage of the dbt project:                model_name                source  \
0           stg_customers  [ecom.raw_customers]   
1           stg_locations     [ecom.raw_stores]   
2         stg_order_items      [ecom.raw_items]   
3              stg_orders     [ecom.raw_orders]   
4            stg_products   [ecom.raw_products]   
5            stg_supplies   [ecom.raw_supplies]   
6               customers                    []   
7               locations                    []   
8   met

CrewOutput(raw='# User Request \nThe user requests to add a new column \'overdue\' to the model `orders` that derives from the `raw_orders` source, without requiring any calculations since this column is already available in the raw data. The user also wants this new column to be accessible in the related `customers` model.\n\n# Validation of Proposed Changes \n\n- For the main model: \n  - **Proposed Changes:**\n    1. Add the \'overdue\' column from `stg_orders` to the `orders` model.\n    \n  - **Validation Status for Each Change:**\n    - Change 1: **Correct**. The \'overdue\' column from `stg_orders` is directly integrated into the `orders` model without calculations.\n    \n  - **Sections of Code Needed to Change:**\n    ```sql\n    WITH orders AS (\n      SELECT *,\n             overdue  -- Add the overdue column\n      FROM {{ ref(\'stg_orders\') }}  -- Source model\n    ),\n    ```\n\n  - **Documentation Changes:**\n    ```yaml\n    models:\n      - name: orders\n        descr

### Test local model

In [35]:
from crewai import LLM, Agent, Task, Crew
#local_llm_name = "Llama-3.2-3B-Instruct-4bit"
local_llm_name = "Qwen2.5-Coder-7B-Instruct-GGUF"
local_llm = LLM(model="lm_studio/"+local_llm_name, base_url="http://127.0.0.1:1234/v1", api_key="asdf")

In [36]:
agent = Agent(
    role="Data Analyst",
    goal="Analyze eCommerce sales data",
    backstory="Expert in data analytics with years of experience",
    llm=local_llm
)

task = Task(
    description="Analyze sales trends from the last quarter and identify key insights.",
    agent=agent,
    expected_output="A detailed report summarizing sales trends, key insights, and recommendations."
)

crew = Crew(
    agents=[agent],
    tasks=[task],
    verbose=True
)

result = crew.kickoff() 
print(result)

ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable


# Agent: Data Analyst
## Task: Analyze sales trends from the last quarter and identify key insights.


# Agent: Data Analyst
## Final Answer: 
---

# Sales Trends Analysis for Q4 2023

## Executive Summary

This report provides a detailed analysis of sales trends from the fourth quarter (Q4) of 2023. The analysis includes key insights, growth metrics, and recommendations to help optimize future sales performance.

### Key Insights

1. **Revenue Growth**: Q4 2023 saw a 5% increase in revenue compared to Q4 2022, driven by strong performances in both online and offline channels.
   
2. **Customer Acquisition Costs (CAC)**: CAC remained stable at $250 per customer, indicating efficient marketing strategies are still effective.

3. **Conversion Rate**: The conversion rate improved by 4% from Q4 2022 to Q4 2023, suggesting that refining the shopping experience has positively impacted user engagement.
   
4. **Top-selling Products**: Products in the electronics and fashion categories showed 

In [37]:
local_flow = dbtChatFlow(files, local_llm)
local_flow.plot()

user_input = "I want to add a new column 'overdue' to the model orders that come from raw_orders source, and have it available in customers. the overdue column is directly available in raw_orders, is not necessairy to calcylate it "
result = local_flow.kickoff(inputs={"request": user_input, "dbt_repo_knowledge_df": dbt_repo_knowledge_df, "vectorstore": loaded_vectorstore, "embedding_function":langchain_openai_embeddings})
display(Markdown(f"<div style='font-size: 18px;'><b>User input:</b> <i>{user_input}</i></div><hr>"))
display(Markdown(result.raw))

ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:Failed to get supported params: 

Plot saved as crewai_flow.html
# Agent: Identify if the user's request explicitly mentions a specific model for retrieving information or implementing changes.
## Task: Verify if the request explicitly mentions a model that requires information retrieval or changes. Request: I want to add a new column 'overdue' to the model orders that come from raw_orders source, and have it available in customers. the overdue column is directly available in raw_orders, is not necessairy to calcylate it  Current dbt lineage of the dbt project:                model_name                source  \
0           stg_customers  [ecom.raw_customers]   
1           stg_locations     [ecom.raw_stores]   
2         stg_order_items      [ecom.raw_items]   
3              stg_orders     [ecom.raw_orders]   
4            stg_products   [ecom.raw_products]   
5            stg_supplies   [ecom.raw_supplies]   
6               customers                    []   
7               locations                    []   
8   met

ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable




# Agent: Identify if the user's request explicitly mentions a specific model for retrieving information or implementing changes.
## Final Answer: 
{
  "status": "DETECTED",
  "identified_model": "stg_orders"
}


# Agent: Locate the most relevant model for the request by analyzing the lineage of models and matching the context provided in the request.
## Task: Locate the models most relevant to the user's request by analyzing lineage and matching the context. User request: I want to add a new column 'overdue' to the model orders that come from raw_orders source, and have it available in customers. the overdue column is directly available in raw_orders, is not necessairy to calcylate it  Main impacted models names: ['stg_orders.sql'] Main impacted models details and code: [Document(metadata={'children': "['order_items', 'orders']", 'has_group_by': False, 'has_select_all_in_last_select': True, 'has_tests': True, 'is_end_model': False, 'is_filtered': False, 'is_macro': '', 'is_seed': '',

ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable


...



# Agent: As a top-level dbt and SQL expert, design robust and efficient solutions to implement changes in the model, ensuring alignment with best practices, optimal performance, and adherence to the project's structure and data modeling principles.
## Final Answer: 
# User Request Summarize the original request concisely.
The user wants to add a new column 'overdue' to the `stg_orders` model, which sources data from `raw_orders`. The 'overdue' column is directly available in `raw_orders`, so no calculation is necessary.

# Proposed Changes to the Main Model - Specify the name of the main model.
- **Model Name:** stg_orders

- **Proposed Changes:**
  - Add a new column `overdue` to the `stg_orders` model.
  - Ensure that all downstream models, particularly the `customers` model, can access and utilize this new column without requiring additional calculations.

## Code Changes:
```sql
WITH SOURCE AS (
    SELECT *
    FROM {{ source('ecom', 'raw_orders') }}
),
renamed AS (
    SELECT
 

ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable




# Agent: As a top-level dbt and SQL expert, design robust and efficient solutions to implement changes in the model, ensuring alignment with best practices, optimal performance, and adherence to the project's structure and data modeling principles.
## Final Answer: 
# User Request
The user wants to add a new column 'overdue' to the `stg_orders` model, which sources data from `raw_orders`. The 'overdue' column is directly available in `raw_orders`, so no calculation is necessary.

# Proposed Changes to Secondary Models
- **Model Name:** customers

- **Proposed Changes:**
  - Add a new column `overdue` to the `customers` model.
  - Ensure that this new column can access the 'overdue' column from the `stg_orders` model without requiring additional calculations.

## Code Changes:
```sql
WITH stg_customers AS (
    SELECT *
    FROM {{ ref('stg_customers') }}
),
stg_orders AS (
    SELECT *
    FROM {{ ref('stg_orders') }}
)
SELECT 
    sc.*,
    so.overdue
FROM 
    stg_customers sc
LEFT

<div style='font-size: 18px;'><b>User input:</b> <i>I want to add a new column 'overdue' to the model orders that come from raw_orders source, and have it available in customers. the overdue column is directly available in raw_orders, is not necessairy to calcylate it </i></div><hr>

# User Request Summarize the user's request briefly and clearly.
The user wants to add a new column 'overdue' to the `stg_orders` model, which sources data from `raw_orders`. The 'overdue' column is directly available in `raw_orders`, so no calculation is necessary. Additionally, the same column should be added to the `customers` model and propagated correctly.

# Validation of Proposed Changes - For the main model:
**Model Name:** stg_orders

**Proposed Changes:**
- Add a new column `overdue` to the `stg_orders` model.
- Ensure that all downstream models, particularly the `customers` model, can access and utilize this new column without requiring additional calculations.

**Validation Status:**
- **Adding the 'overdue' column:** Correct. The code provided is straightforward and does not involve any unnecessary calculations.
- **Documentation update:** Incorrect. The documentation for `stg_orders` should be updated to reflect the correct data type of the `overdue` column if it's a boolean or another specific data type.

**Code Changes:**
```sql
WITH SOURCE AS (
    SELECT *
    FROM {{ source('ecom', 'raw_orders') }}
),
renamed AS (
    SELECT
        id AS order_id,
        store_id AS location_id,
        customer AS customer_id,
        subtotal AS subtotal_cents,
        tax_paid AS tax_paid_cents,
        order_total AS order_total_cents, 
        {{ cents_to_dollars('subtotal') }} AS subtotal, 
        {{ cents_to_dollars('tax_paid') }} AS tax_paid, 
        {{ cents_to_dollars('order_total') }} AS order_total,
        overdue::boolean AS overdue
    FROM SOURCE
)
SELECT *
FROM renamed
```

**Documentation Changes:**
```yaml
{'models': [{'name': 'stg_orders', 
            'description': 'Order data with basic cleaning and transformation applied, one row per order. Includes the \'overdue\' column from raw orders.', 
            'data_tests': [{'dbt_utils.expression_is_true': {'expression': 'order_total - tax_paid = subtotal'}}], 
            'columns': [
                {'name': 'order_id', 'description': 'The unique key for each order.', 'data_tests': ['not_null', 'unique']},
                {'name': 'overdue', 'description': 'Indicates if the order is overdue.', 'data_type': 'boolean'}
            ]}
        ]}        
```

- **Adding the 'overdue' column to `customers`:** Incorrect. The `customer` model should not directly include the `overdue` column from `stg_orders`. Instead, it should aggregate this information appropriately.

# Validation of Proposed Changes - For secondary impacted models:
**Model Name:** customers

**Proposed Changes:**
- Add a new column `overdue` to the `customers` model.
- Ensure that this new column can access the 'overdue' column from the `stg_orders` model without requiring additional calculations.

**Validation Status:**
- **Adding the 'overdue' column:** Incorrect. The `customer` model should not directly include the `overdue` column from `stg_orders`. Instead, it should aggregate this information appropriately.
- **Updating documentation:** Correct. The documentation for `customers` should be updated to reflect the new column.

**Code Changes:**
```sql
WITH stg_customers AS (
    SELECT *
    FROM {{ ref('stg_customers') }}
),
stg_orders AS (
    SELECT *
    FROM {{ ref('stg_orders') }}
)
SELECT 
    sc.*,
    MAX(so.overdue) AS any_overdue_order -- Aggregate to indicate if any order is overdue
FROM 
    stg_customers sc
LEFT JOIN 
    stg_orders so ON sc.customer_id = so.customer_id
GROUP BY 
    sc.*
```

**Documentation Changes:**
```yaml
models: [
    {
        name: 'customers',
        description: 'Customer data with additional information derived from order data, including the \'overdue\' status of orders.',
        columns: [
            {
                name: 'customer_id',
                description: 'The unique key for each customer.',
                data_tests: ['not_null', 'unique']
            },
            {
                name: 'any_overdue_order',
                description: 'Indicates if any order associated with this customer is overdue.'
            }
        ]
    }
]
```

# Required Tests - Include a concise list of tests needed to validate the changes, ensuring correctness and consistency.
- **Check that the 'overdue' column is not completely null:** Ensure that there are rows where `overdue` is not null.

```sql
SELECT COUNT(*)
FROM stg_orders
WHERE overdue IS NOT NULL;
```

- **Check that the number of registers remains consistent:** Verify that the count of rows in the `stg_orders` and `customers` models remains the same before and after the changes.

```sql
-- Before Changes
SELECT COUNT(*) FROM stg_orders;
SELECT COUNT(*) FROM customers;

-- After Changes
SELECT COUNT(*) FROM stg_orders;
SELECT COUNT(*) FROM customers;
```

- **Check that the 'overdue' column in `customers` accurately reflects the data from `stg_orders`:** Ensure that the `any_overdue_order` column is correctly set for each customer based on their order history.

```sql
WITH stg_customers AS (
    SELECT *
    FROM {{ ref('stg_customers') }}
),
stg_orders AS (
    SELECT *
    FROM {{ ref('stg_orders') }}
)
SELECT 
    sc.customer_id,
    MAX(so.overdue) AS any_overdue_order,
    (CASE WHEN MAX(so.overdue) THEN 'Pass' ELSE 'Fail' END) AS test_result
FROM 
    stg_customers sc
LEFT JOIN 
    stg_orders so ON sc.customer_id = so.customer_id
GROUP BY 
    sc.customer_id;
```

# Notes - Ensure all validations and tests are based on the provided context and changes. Avoid including redundant or speculative tests. Be concise and clear, focusing only on critical validations and tests required to confirm the quality of the changes.

These changes should now be validated against the provided tests to ensure that they meet the user's requirements without introducing any errors or inconsistencies.

In [ ]:
import requests
response = requests.get("http://127.0.0.1:1234/v1/models")
print(response.json())  # Verifica el nombre correcto del modelo
